In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from tqdm import notebook
import datetime
import time
import pandas as pd

In [2]:
#드라이버 불러온 후 전체화면 크기로 설정
driver = webdriver.Chrome('./chromedriver.exe')
driver.maximize_window()

### CSV update 코드  
- ```file_name```과 ```header(column)```으로 들어갈 값을 작성

In [3]:
def update_csv(f_name, data_list):
    df = pd.DataFrame(data_list, index=[0])
    df.to_csv(f_name, index=False, encoding = 'utf-8-sig',mode='a')


### jumpit site 크롤링 코드  
- Category 1~21까지 반복하며 나오는 공고들을 하나씩 클릭
-> CSV 형태로 저장 **[col: 회사이름,기술스택, 주요업무, ...]**
- 인터넷 성능에 따라 ```implicityly_wait()```의 시간을 조정하면 될 것 같습니다.

In [4]:
def jumpit_crawling(start_category, start_xpath_location):

    Category_name = [0,'서버/백엔드 개발자','프론트엔드 개발자','웹 풀스택 개발자', '안드로이드 개발자','게임 클라이언트 개발자', '게임 서버 개발자', 'DBA','인공지능/머신러닝', 'devops/시스템 엔지니어',
                     '정보보안 담당자','QA 엔지니어', '개발 PM','HW/임베디드',0,'SW/솔루션','IOS 개발자','웹퍼블리셔','크로스플랫폼 앱개발자','빅데이터 엔지니어','VR/AR/3D','기술지원','블록체인']
    max_category = 22
    max_crawling_data = 1000

    URL = 'https://www.jumpit.co.kr/positions'
    for jobCategory in range(start_category,max_category+1):
        
        if jobCategory == 14:
            continue

        driver.get(URL + f'?jobCategory={jobCategory}') 
        driver.implicitly_wait(time_to_wait=5)
        
        xpath_location = start_xpath_location

        csv_name = f'크롤링_점핏_{jobCategory}.csv'
        df = pd.DataFrame(columns = ['회사명', '지원기간', '공고명', '직무내용', '자격요건', '우대조건', '경력조건', '기술 스택', '출처 URL', '복지'])

        #직무명
        job = Category_name[jobCategory]

        #페이지의 공고들을 반복하여 클릭해서 들어감
        for i in notebook.tqdm(range(max_crawling_data), desc = f'{max_crawling_data}개 크롤링'):
            xpath_location += 1
            driver.implicitly_wait(time_to_wait=5)

            #회사명, 채용공고명
            try:
                company_name = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[1]/section/div[{xpath_location}]/a/div[3]/div').text
                field_recruitment = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[1]/section/div[{xpath_location}]/a/div[3]/h2').text
            
            except:
                df = pd.concat([df, pd.DataFrame(new_row, index=[0])], ignore_index=True)
                df.to_csv(csv_name, index=False)
                break

            #공고 클릭
            driver.find_element(By.XPATH, f'//*[@id="root"]/main/div/div[1]/section/div[{xpath_location}]').click()
            driver.implicitly_wait(time_to_wait=5)

            url = driver.current_url
            skill = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[2]/div/section[2]/dl[1]/dd').text
            qualification = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[2]/div/section[2]/dl[3]/dd').text
            preference = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[2]/div/section[2]/dl[4]/dd').text
            benefit = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[2]/div/section[2]/dl[5]/dd').text
            career = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[2]/div/section[3]/dl[1]').text
            deadline = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[2]/div/section[3]/dl[3]').text

            new_row = {'회사명':company_name,'지원기간':deadline,'공고명':field_recruitment,'직무내용':job,
                       '자격요건':qualification,'우대조건':preference,'경력조건':career,'기술 스택':skill,
                       '출처 URL':url, '복지':benefit}

            df = pd.concat([df, pd.DataFrame(new_row, index=[0])], ignore_index=True)

            #30개에 한 번씩 저장
            if len(df) % 30 == 0:
                df.to_csv(csv_name, index=False)
                print('+30 topic -> CSV file update')

            driver.back()
            driver.implicitly_wait(time_to_wait=3)
        
        if len(df) % 30 != 0:
            df.to_csv(csv_name, index=False)
            print(f'remainder {len(df)%30}  topic -> CSV file update')
            print(f'{job} Category 끝')
            driver.implicitly_wait(time_to_wait=3)

### 데이터 수집 및 시간 측정

In [5]:
#수행해야할 category_number와 xpath_location 입력

start = time.time()
jumpit_crawling(1, 0)
end = time.time()

result = datetime.timedelta(seconds=end-start)
print(result)

+30 topic -> CSV file update
+30 topic -> CSV file update



KeyboardInterrupt: 